note: I consulted on the 'philisophical problems' about the dataset, biases, etc. with Oluwatosin Alliyu but we wrote all of our code separately so we are not submitting as a group. 

<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Homework 1: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner<br/>
<hr style='height:2px'>

In [ ]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2020-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)


## Overview 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Toward this goal, we will address certain question about COVID, and you will scrape data directly from a website. For the remainder of the semester, we will provide you data files directly; however, since real-world problems often require gathering information from a variety of sources, including the Internet, web scraping is a highly useful skill to have.

### Instructions
- To submit your assignment, follow the instructions given in Canvas.

### Learning Objectives
- Get started using Jupyter Notebooks, which are incredibly popular, powerful, and will be our medium of programming for the duration of CS109A and CS109B.
- Become familiar with how to access and use data from various sources (i.e., web scraping and directly from files).
- Gain experience with data exploration and simple analysis.
- Become comfortable with PANDAS as a means of storing and working with data.
- Reflect on what further analysis you may wish to do with this data. For example, given the material we've covered so far, what *more* do you wish you had the ability to do (e.g., modelling, prediction, etc). That is, think about questions you may have about the data, and try to imagine what types of tools you might need to help answer your questions.

### Notes
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. In your code, use a delay of 2 seconds between requests. This helps to not get blocked by the target website -- imagine how frustrating it would be to have this occur. Section 1 of this homework involves saving the scraped web pages to your local machine. Thus, after completing Section 1, you do not need to re-scrape any of the pages, unless you wish to occasionally grab the latest data. 

- Web scraping requests can take several minutes. This is another reason why you should not wait until the last minute to do this homework.
- As you run a Jupyter Notebook, it maintains a running state of memory. Thus, the order in which you run cells matters and plays a crucial role; it can be easy to make mistakes based on *when* you run different cells as you develop and test your code. **Before submitting every Jupyter Notebook homework assignment, be sure to restart your Jupyter Notebook and run the entire notebook from scratch, all at once (i.e., "Kernel -> Restart & Run All")**
- We will be working with COVID data. COVID has impacted everyone in the world, and naturally some people have been greatly more affected than others. We, the teaching staff, are sensitive to this, empathize, and understand that working with COVID data may be unsettling to some. We apologize for any discomfort this may cause. Our intent with this assignment is purely pedagogical, and we'd like to remind students that data science and machine learning can be used to provide insights that can be used for good and invoke change. Toward this goal, parts of the homework are intended to shed light on the unfortunate, widespread inequality that exists. So, while this data may be unsettling, our aim is for the learned skills addressed here -- and in all future assignments -- to provide you with knowledge and confidence to do good work.

## 1. Obtaining Data (17 points)

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

In this exercise, we will focus on automating the downloading of each state's data (via ``Requests``). First, as we will do for every Jupyter Notebook, let's import necessary packages that we will use throughout the notebook (i.e., run the cell below). 

In [ ]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

# NOTE: files will be saved to this directory, so you need to ensure
# that it exists on your system first (it should be visible from the
# directory of where you are running this Notebook file)
# i.e.,
# >> ls
# cs109a_hw1_student.ipynb
# election2016_by_county.csv
# data/
state_dir = "data/"

In [ ]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

<div class='exercise'><b> Exercise 1.1 [1 pt]: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [ ]:
home_page = requests.get(base_url)

<div class='exercise'><b>Exercise 1.2 [2 pts]:</b> In the cell below:
    
- Write a line of code that prints to the screen the status of `home_page` (the web page's returned object). You should receive a code of 200 if the request was successful; then,

- Write code that prints the entire contents of `home_page`</div>


In [ ]:
print('Status Code: {}'.format(home_page.status_code))
print('Webpage Text')
print(home_page.text)

<div class='exercise'><b> Exercise 1.3 [1 pt]:</b>
    
In the cell below, create a new BeautifulSoup object that parses the `home_page` as an HTML document (can be done with 1 line of code)</div>

In [ ]:
soup = BeautifulSoup(home_page.text, 'html.parser')

<div class='exercise'><b> Exercise 1.4 [8 pts]:</b>
    
In the cell below, write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. Feel free to use Regular Expressions, in conjunction with any BeautifulSoup parsing. Specifically, the goal is to populate the `state_urls` dictionary by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

### AS A CRITICAL EXAMPLE:###
Within `state_urls`, one of your <key, value> pairs should be:

``"District of Columbia" : "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia"``

The casing here is **incredibly** important because later, in Exercise 4, you will merge your data with another dataset that has casing of this form. Thus, our key here should be `District of Columbia` and not `District Of Columbia` or `district-of-columbia`.


**NOTES:**
- There are _many_ solutions, but you may find it easiest to use Regular Expression(s)
- Pay attention to the casing example above, so that your later Exercises go smoothly. 
</div>

In [ ]:
state_urls = {}

#find each link set in the html file
for link in soup.findAll('a'):
    
    #get the url
    url = link.get('href')
    
    #check if the url follows the base format
    if re.search(base_url[20:], url):
        
        #if so pull out the state name
        match = re.search(r"state/(.+)", url)
        state_url = base_url + match.group(0)
        
        state = link.text
        
        #add to the dictionary
        state_urls[state] = state_url

Run the cell below to help ensure your formatting is correct and has 51 <key, value> pairs.

In [ ]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

We wish to use the data without having to re-download it every time. So, let's save each webpage to our local hard drive. **NOTE: It's probably okay to download all of the state web pages a few times a day, but it's safer to keep it to a minimum.**

<div class='exercise'><b> Exercise 1.5 [5 pts]:</b>
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. Your job is to make a web request for each URL and save the **contents** out to a file on your hard drive (use `state_dir` as the prefix to the path.) 

**NOTES:**
- Leave the 2 second pause
- You should be saving to a file the actual content of the webpage, not a BeautifulSoup object. That is, you should be able to open the saved files in an editor and see the HTML code, just as you could if you were to view the webpage in your browser and click 'View Page Source'.
- See [official Python documentation](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) for details on how to read/write files to disk
- You should have saved 51 different files to your hard drive.
</div>

In [ ]:
# 1.5 (4 pts) -- save each webpage to disk
for state, url in state_urls.items():
    
    with open(state_dir + state + '.html', 'w') as f:
        f.write(requests.get(url).text)
    
    sleep(2) # LEAVE THIS IN

## 2. Loading and Exploring Data (22 pts)
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it. We want you to understand that PANDAS is a library of useful data structures and operations, but we also wish to remind you that it isn't magic and it isn't the _only_ way to do Data Science; it's just a tool to help, and you could do the same operations without PANDAS. Thus, here we ask you to perform a few operations without using PANDAS, and then in Exercise 3 we will use PANDAS.

**Terminology Notice:** In the United States, every state is comprised of many **counties.** You can think of a **county** as being a pretty large district. 

First, run the cell below to construct `state_info`

In [ ]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

<div class='exercise'><b> Exercise 2.1 [10 pts]: Parsing and storing data</b>
    
Complete the `load_covid_data()` function, which:

- Takes as input `state_info`, which is a list of tuples: (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - \# of confirmed cases (total)
    - \# of deaths
    - \# of confirmed cases (per 100k)
- Stores in a **non-PANDAS** data structure named `covid_data` the above 3 pieces of data, **for every county across every state**
- Returns `county_counts`
    <font color='blue'>

**NOTES:**
- To be clear, as of September 7, 2020, the webpage for Alabama currently lists 67 counties. District of Columbia has 1 county, and Wyoming has 23. Here we are asking you to store in `covid_data` *all counties* across every state. So, later, if we were wished to access just Wyoming's information, you could easily retrieve such for each of its 23 counties, or the info for any of the 67 counties in Alabama. 
- `covid_data` **must not be a PANDAS data structure;** it must use a combination of lists and/or dictionaries. It's up to you to decide how to organize this, e.g., a lists of lists of lists, or a list of dictionaries, or a dictionary of dictionaries, or a dictionary of lists of lists, etc. A guiding decision should be ease of access for computing basic stats (Exercises 2.2, 2.3, and 2.4)
</div>

In [ ]:
# parses all county-level COVID data from the
# passed-in list of (state, filepath) tuples 
def load_covid_data(state_info):
    '''
    input: list of tuples of type (state name, /path/state name)
    output: dictionary of type 
                {State:{"Counties": list, 
                        "Cases": list,
                        "Deaths": list,
                        "Cases (Per 100K)": list}}
            with one entry for each state, and the lists each sorted
            by county
    '''
    
    def load_state_data(state_tuple):
        '''
        input: a tuple from the state_info
        output: an entry to the output dictionary
        
        Includes an assert statement to make sure that the 
        entries in the table are all the same length
        '''
        counties = []
        cases = []
        deaths = []
        cases_norm = []

        with open(state_tuple[1] + '.html') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
        state = state_tuple[0]
        
        #find the counties by looking for links
        for link in soup.find_all('a'):
            url = link.get('href')
            
            #all county links look the same
            if re.search(base_url[20:], url):
                match = re.search("/state/.+/county/(.+)", url)
                county = link.text
                counties.append(county)
                
        #find table entries by looking for a tag
        table = [cases, deaths, cases_norm]
        counter = 0
        for entry in soup.find_all('td'):
            if counter%3 == 2:
                number = float(entry.text.replace(',',''))
            else:
                number = int(entry.text.replace(',',''))
            table[counter % 3].append(number)
            counter += 1 

        county_counts = {"Counties": counties, 
                        "Cases": cases,
                        "Deaths": deaths,
                        "Cases (Per 100K)": cases_norm}
        
        #check that everything is the same length
        assert (len(counties) == 
            len(cases) == 
            len(deaths) == 
            len(cases_norm)), ("The table lengths for " + state +
                                " do not all match up!")
        
        return county_counts
    
    covid_data = {}

    for state_tuple in state_info:
        covid_data[state_tuple[0]] = load_state_data(state_tuple)

    return covid_data

Run the cell below (no changes necessary) to execute your code above

In [ ]:
covid_data = load_covid_data(state_info)

<div class='exercise'><b> Exercise 2.2 [4 pts]: Simple analytics</b>
    
Complete the `calculate_county_stats()` function, which calculates:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide
- These values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).
- If there are ties, return any of the tied counties
</div>

In [ ]:
def calculate_county_stats(covid_data):
    highest = 0
    lowest = 100000000
    
    for state in covid_data:
        stat = covid_data[state]['Cases (Per 100K)']
        max_stat = max(stat)
        min_stat = min(stat)
        if max_stat > highest:
            highest = max_stat
            highest_county_index = stat.index(max_stat)
            highest_county = covid_data[state]['Counties'][highest_county_index]
        
        if min_stat < lowest:
            lowest = min_stat
            lowest_county_index = stat.index(min_stat)
            lowest_county = covid_data[state]['Counties'][lowest_county_index]
        
    print(lowest_county + " has the lowest rate of confirmed COVID cases: " + str(lowest) + " per 100k")
    print(highest_county + " has the highest rate of confirmed COVID cases: " + str(highest) + " per 100k")

Run the cell below (no changes necessary) to execute your code above

In [ ]:
calculate_county_stats(covid_data)

<div class='exercise'><b> Exercise 2.3 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths()` function, which calculates:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- These values you report should be Integers, not Floating point numbers.
- If there are ties, return any of the tied counties

</div>

In [ ]:
def calculate_state_deaths(covid_data):
    highest = 0
    lowest = 10000000
    for state in covid_data:
        stat = sum(covid_data[state]['Deaths'])
        if stat > highest:
            highest = stat
            highest_state = state
        
        if stat < lowest:
            lowest = stat
            lowest_state = state
 
    print(lowest_state + " has the fewest COVID deaths: " + str(lowest))
    print(highest_state + " has the most COVID deaths: " + str(highest))

Run the cell below (no changes necessary) to execute your code above

In [ ]:
calculate_state_deaths(covid_data)

<div class='exercise'><b> Exercise 2.4 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate()` function, which calculates:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- To calculate a state's population, we are asserting that is sufficient to sum the population over all counties, and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- **If a county has reported 0 COVID cases,** then we should ignore this county as we estimate its county population. Thus, that county would contribute 0 to its state population total.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [ ]:
def calculate_state_deathrate(covid_data):
    highest = 0
    lowest = 10000000
    for state in covid_data:
        base = covid_data[state]
        state_pop = 0
        state_deaths = 0
        i = 0
        
        for i in range(len(covid_data[state]["Counties"])):
            if base["Cases (Per 100K)"][i] != 0:
                county_pop = base["Cases"][i]/base["Cases (Per 100K)"][i] * 100000
                state_pop += county_pop
                state_deaths += base["Deaths"][i]
        
        death_rate = state_deaths/state_pop
        stat = death_rate
        
        if stat > highest:
            highest = stat
            highest_state = state
        
        if stat < lowest:
            lowest = stat
            lowest_state = state
    
    print(lowest_state + " has the lowest COVID death rate; 1 out of every " + str(int(1/lowest)) + " people has died")
    print(highest_state + " has the highest COVID death rate; 1 out of every " + str(int(1/highest)) + " people has died")

Run the cell below (no changes necessary) to execute your code above

In [ ]:
calculate_state_deathrate(covid_data)

## 3. PANDAS (36 pts)
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As you saw in the above exercises, doing the most basic analytics is possible, but it can quickly become cumbersome. As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

<div class='exercise'><b> Exercise 3.1 [10 pts]: Converting to PANDAS</b>

In Exercise 2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

Complete the `convert_to_pandas()` function, which converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 5 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# covid cases per 100k (Integer)
    - \# covid deaths (Integer)
- The columns should be titled **exactly** as listed above

**NOTE:**
- If there exists multiple counties with the same name, each of which belonging to a different state, then there should be a distinct row for each.
- The 3 columns that correspond to COVID counts should all be Integers (e.g., 1498), not Floating point digits (e.g., 1498.0)
</div>

In [ ]:
def convert_to_pandas(covid_data):
    neat_data = {}
    index = 0
    for state in covid_data:
        d = covid_data[state]
        for i in range(len(d["Counties"])):
            neat_data[index] = {'county': d["Counties"][i],
                               'state': state,
                               '# total covid cases': d["Cases"][i],
                               '# covid cases per 100k': d["Cases (Per 100K)"][i],
                               '# covid deaths': d["Deaths"][i]}
            index += 1
    
    covid_df = pd.DataFrame(neat_data).T
    covid_df = covid_df.astype({'county': object,
                               'state': object,
                               '# total covid cases': int,
                               '# covid cases per 100k': float,
                               '# covid deaths': int})
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [ ]:
covid_df = convert_to_pandas(covid_data)
covid_df

<div class='exercise'><b> Exercise 3.2 [5 pts]: Simple analytics</b>

Complete the `calculate_county_stats2()` function, **which should obtain identical information as problem 2.2, but now using the PANDAS `covid_df` DataFrame.**

That is, it should calculates:
1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).

</div>

In [ ]:
def calculate_county_stats2(covid_df):
    column = '# covid cases per 100k'
    
    lowest = covid_df[covid_df[column] == min(covid_df[column])].iloc[0]
    highest = covid_df[covid_df[column] == max(covid_df[column])].iloc[0]

    print(lowest['county'] + " (" + lowest['state'] + ") has the lowest rate of confirmed COVID cases: " + str(lowest[column]) + " per 100k")
    print(highest['county'] + " (" + highest['state'] + ") has the highest rate of confirmed COVID cases: " + str(highest[column]) + " per 100k")

Run the cell below (no changes necessary) to execute your code above

In [ ]:
calculate_county_stats2(covid_df)

<div class='exercise'><b> Exercise 3.3 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths2()` function, **which should obtain identical information as problem 2.3, but now using the PANDAS `covid_df` DataFrame.**
1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Integers, not Floating point numbers.
</div>

In [ ]:
def calculate_state_deaths2(covid_df):
    column = '# covid deaths'
    
    group = covid_df.groupby(by='state').sum()
    low_state = group[group[column] == min(group[column])].head(1).index[0]
    high_state = group[group[column] == max(group[column])].head(1).index[0]
    lowest = min(group[column])
    highest = max(group[column])
    print(low_state + " has the fewest COVID deaths: " + str(lowest))
    print(high_state + " has the most COVID deaths: " + str(highest))

Run the cell below (no changes necessary) to execute your code above

In [ ]:
calculate_state_deaths2(covid_df)

<div class='exercise'><b> Exercise 3.4 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate2()` function, **which should obtain identical information as problem 2.4, but now using the PANDAS `covid_df` DataFrame.**

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- Just as in, 2.4, to calculate a state's population, we are asserting that is sufficient to sum the population over all counties -- and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- Just as in 2.4, counties with 0 COVID cases should contibute 0 to the total population of the state.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [ ]:
def calculate_state_deathrate2(covid_df):
    covid_df_no0 = covid_df[covid_df['# total covid cases'] != 0]
    covid_df_no0.loc[:,'population'] = covid_df_no0['# total covid cases']/covid_df_no0["# covid cases per 100k"]*100000
    states = covid_df_no0.groupby(by = 'state').sum()
    states['death rate'] = states['population']/states['# covid deaths']
    
    lowest = states[states['death rate'] == max(states['death rate'])].iloc[0]
    highest = states[states['death rate'] == min(states['death rate'])].iloc[0]

    print(lowest.name + " has the lowest COVID death rate; 1 out of every " + str(int(lowest['death rate'])) + " people has died")
    print(highest.name + " has the highest COVID death rate; 1 out of every " + str(int(highest['death rate'])) + " people has died")

Run the cell below (no changes necessary) to execute your code above

In [ ]:
calculate_state_deathrate2(covid_df)

These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

<div class='exercise'><b>Exercise 3.5 [5 pts]: Further analytics</b>
    
Complete the `add_death_stats()` function below, which should add 2 new columns:
- `case_fatality_rate` and
- `# covid deaths per 100k`

And return the updated DataFrame **sorted by case_fatality_rate in ascending order** 

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# total covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate
- DataFrame should be sorted by `case_fatality_rate` in ascending order
- Again, the values for `case_fatality_rate` should be < 1. A value of 1 would mean that 100% of people who tested positive for COVID also died.
- `# covid deaths per 100k` is simply defined as the # of COVID deaths for every 100,000 people. We calculate this on a per-county basis.
</div>

In [ ]:
def add_death_stats(covid_df):
    # get rid of counties with no cases as we cannot estimate their population
    covid_df = covid_df[covid_df['# total covid cases'] != 0]
    
    d = covid_df
    
    d.loc[:,'population'] = d['# total covid cases']/d["# covid cases per 100k"]*100000
    d.loc[:,'case_fatality_rate'] = d['# covid deaths']/d['# total covid cases']
    d.loc[:,'# covid deaths per 100k'] = d['# covid deaths']/(d['population']/100000)
    
    #sort code by ascending fatality
    d.sort_values(by = 'case_fatality_rate', inplace = True)
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [ ]:
covid_updated = add_death_stats(covid_df)
covid_updated

<div class='exercise'><b> Exercise 3.6 [6 pts]: Reflection:</b> Data Analysis allows us to better understand a system or scenario.
</div>

<div class='exercise'><b>Exercise 3.6: Q1 (2  of 6 pts)</b>
    
Having looked at the results from Exercises 3.3, 3.4, and 3.5, what are some trends you've noticed and any conclusions you have? (2-3 sentences)?</div>

Regardless of the statistic we are looking at - case fatality, total deaths, case rate, death rate - there are huge variations accross the country, with some areas being affected much more than other areas. This leads me to believe that there are other factors that render our earlier assumption that rates such as fatality or death rate should be consistent accross locations incorrect. 

YOUR RESPONSE HERE

<div class='exercise'><b>Exercise 3.6: Q2 (2 of 6 pts)</b>
    
Having looked at the results from Exercise 3.5 (i.e., `covid_updated` DataFrame), do you think the original data is reliable and accurate? Are there any potential biases that you're aware of or concerned about? Please explain (3-5 sentences).</div>

We threw out all of the counties that did not have any cases reported, which will certainly bias our answers. Instead it would be a better idea to get the county populations from another source instead of using the cases per 100k to calculate the population

I do not think that our calculation of case fatality rate makes sense to compare accross counties because we know for a fact that not all counties are conducting testing at the same rate. I think death/population is a much better number to consider. Similarly, I think the 'total deaths' is a much more reliable number to consider than 'total cases'. 

YOUR RESPONSE HERE

<div class='exercise'><b>Exercise 3.6: Q3 (1 of 6 pts)</b>
    
If a county has 15 confirmed deaths, how many cases would you expect? What would you expect its population to be? Explain why (1-2 sentences in total)?

**NOTE:** For this question, we aren't evaluating the accuracy of your answer but your thought-process and reasoning.
</div>

1) Check if the stats on average deaths per 100K and fatality rate differ significantly between all conties and counties with about 15 deaths

In [ ]:
# all counties

covid_updated.loc[:, ['case_fatality_rate', '# covid deaths per 100k']].describe()

In [ ]:
filtered = covid_updated[
    (covid_updated['# covid deaths']<18) & 
    (covid_updated['# covid deaths']> 12)]
filtered.loc[:, ['case_fatality_rate', '# covid deaths per 100k', 'population', '# covid deaths']
            ].describe()

It looks like the distribution has a long tail since the standard deviation is so large compared to the mean in both cases. 

Since this is the case I will use the median instead of the mean since it is more robust to outliers. 

2) Looking at the median, the data pulled from counties with around 15 deaths does significantly vary from the overall data, and there are ~200 counties represented, so I am comfortable drawing conclusions from that smaller dataset.

3) Find expected cases and population. 

**Expected population is ~34k** (found from the median population of the filtered data set)

**Expected case number is 625** (found by 15/median fatality rate of the filtered data set)

YOUR RESPONSE HERE

<div class='exercise'><b>Exercise 3.6: Q4 (1 of 6 pts)</b>
    
What further questions do you wish to answer about COVID, including ones that may not be possible to answer from this data alone (e.g., Is there a correlation between the average age of people in a county and the # of COVID deaths)? Write at least 3 of your questions.</div>

- Is there a correlation between county size and a larger standard deviation an these statistics? i.e are the small counties a source of variability in our data due to small numbers?

- Is there a correlation between county wealth and outcomes?

- Is there a correlation between nearby hospitals/medical facilities and outcomes for the counties?

YOUR RESPONSE HERE

## 4. MORE DATA (25 pts)
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2016_by_county.csv`.

<div class='exercise'><b>Exercise 4.1 [4 pts]: Load more data</b>

Complete the `merge_data()` function, which should:
1. First, load `election2016_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 7 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- case_fatality_rate

along with these 14 columns from `election2016_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- votergap
- trump
- clinton

**NOTES:**
- We are dropping two columns from `election2016_by_county.csv`:
    - fipscode
    - population
- Do not attempt to manually fix any of the state or county names. That is, **our merging should require the state and county names to be identical (case-sensitive) between the two DataFrames.** If there is a discrepancy between the two, do not worry about adjusting these names to find a perfect match.

**HINT:** there are many ways to solve this, but you may find the [pandas.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function can be really helpful
</div>

In [ ]:
def merge_data(covid_updated, filepath):
    
    filepath = 'election2016_by_county.csv'
    csv = pd.read_csv(filepath, usecols = ['state', 'county', 'hispanic', 'minority',
           'female', 'unemployed', 'income', 'nodegree', 'bachelor', 'inactivity',
           'obesity', 'density', 'cancer', 'votergap', 'trump', 'clinton'])
    
    return covid_updated.merge(csv, on = ['state', 'county'])

Run the cell below (no changes necessary) to execute your code above

In [ ]:
merged = merge_data(covid_updated, 'election2016_by_county.csv')

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

<div class='exercise'><b>Exercise 4.2 [5 pts]: Data Construction / Understanding</b>
</div>

<div class='exercise'><b>Exercise 4.2: Q1 (1 of 5 pts)</b>
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`? Running the cell below should print to the screen your answer.
</div>

In [ ]:
print("{} rows were lost during merge.".format(covid_updated.shape[0]-merged.shape[0]))


<div class='exercise'><b>Exercise 4.2: Q2 (2 of 5 pts)</b>  

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. Running the cell below should print to the screen your answer.
</div>

In [ ]:
covid_updated[~covid_updated['county'].isin(merged['county'])]

<div class='exercise'><b>Exercise 4.2: Q3 (2 of 5 pts)</b>  
? If we needed to be highly thorough and needed comprehensive data coverage, do you have any suggestions on how we could quickly, soundly fix most or all of them? (Write 2-3 sentences.)
    
<b>NOTE: Please do not actually fix these mismatches; for this Exercise, it's okay that the `merged` DataFrame is smaller than `covid_updated`</b>
</div>

It looks like most of the counties that were not included had weird punctuation like dashes or were formatted as 'Williamsburg, City of'.

We could check if the standard format in the .csv is 'City of Williamsburg' and if so use regex searching to find these cases and standardize their format.

This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

<div class='exercise'><b>Exercise 4.3 [2 pts]: Cleaning the data</b>

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information. Write code in the cell below to update the `merged` DataFrame so that all rows with 0 deaths are removed.

In [ ]:
merged = merged[merged['# covid deaths']>0]

Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [ ]:
merged.describe()

Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

<div class='exercise'><b>Exercise 4.4 [3 pts]: Partitioning our data</b>
    
Complete the `partition_df()` function, which takes as input:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

For example, if we called `partition_df(merged, 'obesity', 30, 45)`, it should return a subset of the `merged` DataFrame that has obesity values between 30 and 45 (and including the boundary values of 30 and 45).
</div>

In [ ]:
def partition_df(df, column_name, min, max):
    return df[(df[column_name] >= min) & 
              (df[column_name] <= max)].loc[:,['# covid cases per 100k',
                                            'case_fatality_rate',
                                            '# covid deaths per 100k']]

<div class='exercise'><b>Exercise 4.5: [4 pts] Exploratory Data Analysis</b>
    
Identify a few features that you're interested in, and inspect if there's any correlation with the COVID data. Specifically, simply run your `partition_df()` function below, many times, each with a different subset of the data -- select a range of values and a particular feature. For example, if I'm interested in __cancer__, I could look at the 4 quartiles (per `.describe()`) and use those ranges of values as I repeatedly execute `partition_df()`. For this exercise, after running the function several times, **write 3-5 sentences about any patterns or correlations you noticed or didn't notice but expected to find.**
</div>

In [ ]:
merged.dtypes

**Income**

In [ ]:
var = merged['income'].describe()
var

look at bottom vs top quartile

In [ ]:
partition_df(merged, 'income', var.loc['min'], var.loc['25%']
            ).describe().loc[['count','mean', '50%'],:]

In [ ]:
partition_df(merged, 'income', var.loc['75%'], var.loc['max']
            ).describe().loc[['count','mean', '50%'],:]

For income levels, the fatality rate is about the same, but the deaths per 100k is 2x more for low income areas. 

So this implies that the testing regimin was about the same regardless of income bracket, but poorer counties were hit harder. 

**Obesity**

In [ ]:
var = merged['obesity'].describe()
var

In [ ]:
partition_df(merged, 'obesity', var.loc['min'], var.loc['25%']
            ).describe().loc[['count','mean', '50%'],:]

In [ ]:
partition_df(merged, 'obesity', var.loc['75%'], var.loc['max']
            ).describe().loc[['count','mean', '50%'],:]

The same trends are true of areas with higher obesity, although the difference between the two quartiles is less pronounced than with income

`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

Run the cell below.

In [ ]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

<div class='exercise'><b>Exercise 4.6 [5 pts]: Aggregate data</b>
    
    
Write code in the cell below to group (and display) the data according to the 4 income groups. Also, while we will still keep the same columns (i.e, features), the values of each should now represent the __average__ value of all rows that were subsumed in the making of the aggregate income-group. Your resulting DataFrame should have just 4 rows (income-group-1, income-group-2, income-group-3, income-group-4). See example in the cell below.


Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [ ]:
# EXAMPLE: If our `merged` DataFrame were
# COUNTY    INCOME GROUP    BACHELOR ... (other columns, too)
#   A            2             50
#   B            1             20
#   C            1             30
#   D            2             70
#   E            3             95

# it should become
# INCOME GROUP    BACHELOR ... (other columns, too)
#   1                25
#   2                60
#   3                95

merged.groupby(by='income group').mean()

<div class='exercise'><b>Exercise 4.7 [2 pts]: Conclusions</b>
</div>

<div class='exercise'><b>Exercise 4.7: Q1 (1 of 2 pts)</b>
What are your conclusions/finding from this alternative view of the data? (2-4 sentences).
</div>

- Being in the lowest income group has much worse outcomes. Between the other three groups, the highest income group does have better outcomes, but the gap is largest between the bottom and the 2nd lowest.

- All of the income brackets had about the same case fatality rate

- some of the other variables, such as 'density', and 'bachelor' correlate well to income, while others such as 'female', 'inactivity', and 'cancer' do not

<div class='exercise'><b>Exercise 4.7: Q2 (1 of 2 pts)</b>
What are some weaknesses from this view of the data? (2-4 sentences).
</div>

- Using the mean instead of the median seems like a weakness to me given that the data is skewed and with a large standard deviation.

- We are using fairly large groups. Since there is such a big gap between the bottom quartile and the third quartile, I wonder where the 'tipping point' of income is. 

- It is hard to tell which variables have the strongest effect

## Moving Forward

In this homework assignment, we've focused on gathering, parsing, and exploring data. However, what if we wanted to *predict* some behavior of the data. For example, imagine one is curious how a particular county will respond to COVID. Or, imagine we looked at counties' COVID data on a weekly basis, one could be interested in predicting the upcoming week's behavior.

Alternatively, one could be interested in *inference*, whereby we are more concerned with trying to understand __why__ and __how__ a system behaves the way it does. We might wish to understand which factors most correlate and cause a certain event to happen. This could give us insights into where certain inequalities persist.

For both *prediction* and *inference*, our computational method of solving such a task is referred to as a model. For the remainder of CS109, we will spend significant focus on various models.
</div>

## Reflection

As a reminder, this is just **one** of the homework assignments in this course, the point of which is to assess your learning and to provide both you and us with an indication as to how aligned your knowledge and skills are with our learning objectives. To this end, we encourage you to reflect on your progress, strengths, and weaknesses and to make changes, if necessary, to accomplish your goals. Likewise, please reach out to the TFs and teaching staff if you need help. We want everyone to feel comfortable in being honest about these elements, with both herself/himself and us. For these purposes, we will ask you several times throughout the semester to complete an anonymous poll.